# Hypothesis Testing: Binomial Testing


Notes (Hook):

Anyone here put milk in your tea? Is this a popular thing to do in
Canada? My family usually doesn't put milk in our tea, but there's a
pretty famous scenario from statistics that goes something like this.


## Scenario

It's the 1920's and Ron and Muriel are coworkers. Muriel swears that when
she drinks tea and milk together, she can taste whether the milk was
poured in first or second (and she prefers the milk go in first in her
tea).

Ron is unconvinced (he thinks it shouldn't matter at all what order the
tea and milk enter the cup).

What kind of proof could satisfy Ron's skepticism?
